### Two Pointers

<b>141.Linked List Cycle </b>. 

fast pointer and slow pointer.  
fast pointer goes two steps at a time. slow pointer goes one step at a time. When they enter the loop, the distance between two pointers will shrink one per time, so finally they will meet.  
快指针每次走两步，慢指针一次走一步。 在慢指针进入环之后，快慢指针之间的距离每次缩小1，所以最终能相遇。

In [2]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, x):
#         self.val = x
#         self.next = None

def hasCycle(head):
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            return True
    return False

<b>142. Linked List Cycle II</b>  
a good explanation [here] (https://leetcode.com/problems/linked-list-cycle-ii/discuss/44781/Concise-O(n)-solution-by-using-C%2B%2B-with-Detailed-Alogrithm-Description) also read the notes in evernotes. 
first step: find where fast pointer and slow pointer meet.
second step: keep where slow pointer stands, move fast pointer to the head, both pointer take one step at a time, the point they meet is the start of the circle => approved mathematically
2 (L1 + L2 ) = (L1 + L2 + n*C) => L1 = (n-1)c + (c-L2).   n is how many time the circle is looped, usually 1
C is distance of the whole loop 
L1: distance between head to start point of loop  
L2: distance between start point to meeting point inside of the loop
distance that slow pointer traveled before they meet: (L1 + L2 )
distance that fast poitner traveled before they meet: (L1 + L2 + n*C)

In [3]:
def detectCycle(head):
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:       
            fast = head
            while slow!= fast:
                slow = slow.next
                fast = fast.next
            return slow
    return 

<b> 287. Find the Duplicate Number</b>.   
the underlining question is to find if the list contains a loop  
using the value as index for potential chaining.  
index 0 as the head, nums[0] as the next index, and so on    
Example 0=>1=>3=>2=>4=>2.  So 2 is what it should return. Slow initiates at 1, fast initiates at 3, 2 is where the loop begin, 0 is the head

In [4]:
def findDuplicate(nums):   
    if not nums:
        return False
    # ideally initialize slow = fast = head, which is 0 here, but when situation is not allowed, can initiate
    # slow to head.next (which is nums[0]), fast to head.next.next (nums[nums[0]])
    slow = nums[0]      
    fast = nums[nums[0]]
    while slow != fast:
        slow = nums[slow]
        fast = nums[nums[fast]]
    fast = 0    # index 0 is the head, reset fast to the head
    while fast!=slow:
        fast = nums[fast]
        slow = nums[slow]
    return slow   

<b>80.Remove Duplicates from Sorted Array II </b>

my way: check number k steps way, if current num == num k step away, pop the k step away number out of list, else move current pointer forward,   
Other way: using two pointers, I is one to keep track of where next valid number should be put, also n which is loop the number of the array, compare to I-2 position.

In [9]:
def removeDuplicates(nums):
#     n = len(nums)
    i=0
    while i < len(nums)-2:  # calculating len(nums) every time instead of saving it as n, because after pop the len has changed
        if nums[i]==nums[i+2]:
            # using pop is not efficient!!
            nums.pop(i+2)
        else:
            i+=1
    return len(nums)
            

In [10]:
nums = [1,1,1,2,2,3]
removeDuplicates(nums)

5

In [11]:
def removeDuplicates(nums):   # better way!
#     n = len(nums)
    i = 0  # first pointer, index, keep track of NEXT position to put next valid number
    for n in nums:   # second pointer is the actual number of the array
        if (i<2) or (n>nums[i-2]):
            # i < 2 : for the first 2 elements of nums, keep as what they are
            # n > nums[i-2]: if current number > second last number in the valid array, meaning n can be the next valid number
            nums[i] = n
            i += 1
    return i
    

In [12]:
nums = [1,1,1,2,2,3]
removeDuplicates(nums)

5

<b>146.LRU Cache </b>  
Least Recently Used (LRU). 
basic idea
using double linked list structure to keep track of the order, when new item come, always keep it at tail (head), in this way the least used item is always the closest to head (tail) one.  
to implement the double linked list, can do in from scratch, like <b>way 1 </b>  
or do it using python builtin type: OrderedDict in <b> way 2 </b>, in the doc's example, it is actually used in the LRU cases



<b>way 1</b>

In [13]:
class Node:
    """double linked list"""
    def __init__(self, k, v):
        self.key = k
        self.val = v
        self.prev = None
        self.next = None

class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.head = Node(0, 0)  # can use 0 because assuming valid value is always positive, meaning > 0
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head
        self.cache = {}  # a dict of double linked node instance
    
    def _remove(self, node):
        """get rid of the linkage will delete the node"""
        prev_node = node.prev
        next_node = node.next
        prev_node.next = next_node
        next_node.prev = prev_node
    
    def _add(self, node):
        """
        add the node to the tail of the double linked list,
        in this way the node near the head will be the least used node, can choose to always add to the head as well, in this way the least used on is closet to the tail
        """
        prev_node = self.tail.prev
        node.prev = prev_node
        node.next = self.tail
        prev_node.next = node
        self.tail.prev = node
        

    def get(self, key: int) -> int:
        if key in self.cache:
            node = self.cache[key]
            self._remove(node)
            self._add(node)  # to make it always at the tail, hence ordered
            return node.val
        return -1
            


    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            # no need to check capacity
            node = self.cache[key]
            self._remove(node)
        node = Node(key, value)
        self._add(node)  # add to the double linked list
        self.cache[key] = node # add to the cache dict
        
        if len(self.cache) > self.capacity:
            # always remove the node closet to head
            del_node = self.head.next
            self._remove(del_node)
            del self.cache[del_node.key]
            
                
# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

<b> way 2</b>  
OrderedDict: https://docs.python.org/3/library/collections.html#collections.OrderedDict

In [14]:
from collections import OrderedDict
class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = OrderedDict()
        


    def get(self, key: int) -> int:
        if key in self.cache:
            self.cache.move_to_end(key) # to keep its order high up, move item to the end
            return self.cache[key]
        else:
            return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            del self.cache[key]
        self.cache[key] = value
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)  # FIFO



# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

<b>42.Trapping Rain Water </b>

two pointers, keep track of left max and right max value. while height[left] and height[right] is current value. when left max <= right max, the water can be trapped is determined by the ledt side, which is left_max - height[left]. vice versa, when left_max > right_max, using the right part to calculat the water amount.

Attention: while left <= right, equal sign will make sure cases like [2, 0, 2] will get chance to calcuate the 2-0 case.

In [15]:
def trap( height):
    """four pointers"""
    left_max, right_max = 0, 0
    left, right = 0, len(height)-1
    result = 0
    while left <= right: # why <=, because case like [2,0,2] wont get chance to calculate the 2-0
        if left_max<=right_max:
            # water amount is dependent on left side
            left_max = max(left_max, height[left])
            result += (left_max - height[left])
            left+=1
        else:
            # water amount is dependent on right side
            right_max = max(right_max, height[right])
            result += (right_max - height[right])
            right -= 1
    return result

In [16]:
height= [ 2, 0, 2]
trap( height)

2

<b>11.Container with Most Water </b>  
somehow similar to the <b>trapping rain water</b> question above. But still different. Trapping water calculates the sum of each hole, so need left_max and right_max to hold the bar also use them to determine where to move.   
This question no need the left_max and right_max, but having the same idea of comparing the current left and right height to determine which direction to move, try to get the max of the container area.  
if left bar < right bar:   
moving right bar wont help to increase the area, since the height of container is determined by the shorter bar, which is left bar, and the x axis length is shrinking because of the moving. So moving the left bar is possible to increase the area because it may pointe to a longer bar that increase the area enougth to offset the shrink brought by smaller x-axis.
if left bar == right bar:
moving to either direction is okay
if left bar > right bar:   
moving the right bar, same reason as above, because moving left bar wont bring any better result, but moving right bar might will





In [41]:
def maxArea(height):
    n = len(height)
    l, r = 0, n-1
    result = 0
    while l < r:
        result =max(result, (r-l)*min(height[l], height[r]))
        if height[l] <= height[r]:
            # moving the r pointer wont help anything, move left may encouter a higher bar that add back the area losed by shorter x-axis length
            # if left height and right height are the same, move either one is fine
            l+=1
        else:
            r-=1
    return result

In [42]:
height = [1,8,6,2,5,4,8,3,7]
maxArea(height)

49

<b>5.Longest Palindromic Substring </b>

way 1: O(N^2)

In [20]:
def longestPalindrome(s):
    result = ""
    for i in range(len(s)):
        # odd number case
        subs = helper(s, i, i)
        if len(subs)>len(result):
            result = subs
        # even number case
        subs = helper(s, i, i+1)
        if len(subs) > len(result):
            result = subs

    return result

def helper(s, l, r):
    """
    l, r are left and right pointers, returns the palindromic str
    l and r start at same location when considering odd num case
    l and r start at the adjacent location for even num case
    l goes left, r goes right, until while case not meet

    """

    while (l>=0) and (r<len(s)) and (s[l] == s[r]):
        l-=1
        r+=1
    return s[l+1: r]  # substring from l+1 to r-1, the last valid index

In [21]:
s = "babad"
longestPalindrome(s)

'bab'

<b>15. 3Sum </b>

This [post](https://leetcode.com/problems/3sum/discuss/232712/Best-Python-Solution-(Explained)) very well explained the O(N^2) solution. It is a two pointers solution. Actually three pointers.  
Sorting takes O(NlogN)  
We iterate through the nums once, and each time we iterate the whole array again by a while loop  
So it is O(NlogN+N^2)~=O(N^2)  

There are some smart thougts here
if starting number > 0, then no need to continue, cause all positive numbers never meet the criteria  
if number in index i is the same as previous number, then continue, to avoid duplicated cases  
when case meet, conduct another 2 while loop to eliminate duplicated left and right pointers value to eliminate duplicated cases

In [3]:
def threeSum(nums):
    nums.sort()
    result = []
    n = len(nums)
    # not need to consider the last two elements as i, because no room for l and r (extra two values)
    for i in range(n-2): 
        if nums[i] > 0: break  # positive numbers wont meet the Q
        # same value no need to consider again, because need to find Unique triplets
        if i > 0 and nums[i]==nums[i-1]: continue
        l = i+1  # left pointer only need to start after i
        r = n-1  # right pointer always start at the end of array
        while l < r:
            total = nums[i]+nums[l] + nums[r]
            if total < 0:
                # too small, should be larger
                l+=1
            elif total > 0:
                # too big
                r-=1
            else:
                result.append([nums[i],nums[l],nums[r]])
                # to avoid getting repeated result
                while l < r and nums[l]==nums[l+1]: l+=1
                while l < r and nums[r]==nums[r-1]: r-=1
                # suppose we never enter the while loop above, still need to move forward. 
                # the l out of the loop still have the same value of the nums[l] before the loop, so still need to move forward
                l+=1
                r-=1
    return result

In [23]:
nums = [-1, 0, 1, 2, -1, -4]
threeSum(nums)

[[-1, -1, 2], [-1, 0, 1]]

#### some similar questions to 3Sum  
<b>1.two sum </b>

Array is not sorted, and sorting in advance wont help either because it wants to return the oringial index. so cant use two pointers. Using hashMap to store the value as key and its index as val, and loop once can solve the prolbme.  
Time complexity: O(N)

In [24]:
def twoSum(nums, target):
    """
    two pointers
    is the array sorted?
    """

    hash_map={}
    for ind, n in enumerate(nums):
        val = target - n
        if val in hash_map:
            return [hash_map[val], ind]
        hash_map[n]=ind
    # not returning False because the Q assume there alwasy exist an unique answer

In [25]:
nums = [2, 7, 11, 15]
target = 9

twoSum(nums, target)

[0, 1]

<b>167.Two Sum II - Input array is sorted </b>

way 1: using the hashmap way as twosum above

In [4]:
def twoSum(numbers, target):
    hash_map = {}
    for ind, num in enumerate(numbers):
        if num in hash_map:
            return [hash_map[num]+1, ind+1]
        else:
            hash_map[target-num] = ind
    

In [5]:
numbers = [2,7,11,15] 
target = 9
twoSum(numbers, target)

[1, 2]

way 2:  
using two pointers. start from left and right, moving to the right direction  
this is what is used in the <b>3 Sum question</b> too

In [26]:
def twoSum(numbers, target):
    i, j = 0, len(numbers)-1
    while i < j:
        val = numbers[i] + numbers[j]
        if val  < target:
            i+=1
        elif val > target:
            j-=1
        else:
            return [i+1, j+1]

In [32]:
numbers = [2,7,11,15] 
target = 9
twoSum(numbers, target)

[1, 2]

<b>16. 3sum closest </b>  
same as 3sum question, first need to sort the array. 3 pointers, move l and r based on sum and target  
but the actual returned value is min distance  
which is also O(N^2)

In [34]:
def threeSumClosest(nums, target):
    n = len(nums)
    result = float("inf")
    nums.sort()
    for i in range(n-2):
        # assume only have exactly one solution
        l = i+1
        r = n-1
        while l < r:
            val = nums[i]+nums[l]+nums[r]
            if val > target:
                r-=1                 
            elif val < target:
                l+=1
            else:
                return val
            if abs(result-target) >  abs(val-target):
                result = val
    return result

In [35]:
nums = [-1,2,1,-4]
target = 1
threeSumClosest(nums, target)

2

<b>259. 3Sum Smaller </b>

In [36]:
def threeSumSmaller(nums, target):
    """tricky"""
    nums.sort()  # okay to sort, the i<j<k just mean no duplicate usage, and the answer just count
    n = len(nums)
#     if n <=2:
#         return 0
    result = 0
    for i in range(n-2):
        l = i+1
        r = n-1
        while l < r:
            if nums[i]+nums[l]+nums[r]<target:
                result += (r-l)  # key point, otherwise will miss cases [i, l, between l and r], since r only updates when sum >= target
                l +=1
            else:
                r -=1
    return result

In [37]:
nums = [-2,0,1,3]
target = 2
threeSumSmaller(nums, target)

2

<b> 3. Longest Substring without repeating characters </b>  
keep a hash map to track all visited characters, the search in a hashmap is O(1). char as key, index as value.  
two pointers to construct a <b> sliding window </b>, i is the start of the substring, j keeps moving forward. if j meets a used char in hashmap, i starts to moving forward and updating the hashmap by removing the chars until deleting the duplicated char that j now points. now i is at right of the last found duplicated char.  
the char between i and j have already checked no duplicates, so j just keep moving forward, no need to re-check chars between new i and j.

The two pointers only move forward
O(2n) = 2(N)

In [39]:
def lengthOfLongestSubstring(s):
    n = len(s)
    i, j = 0, 0
    result = 0
    temp = {}   # visisted chars, char as key, index as value
    while j < n:
        if s[j] not in temp:
            temp[s[j]] = j
            j+=1
            result = max(result, j-i)
        else:
            del temp[s[i]]
            i += 1
    return result

In [40]:
s="abcabcbb"
lengthOfLongestSubstring(s)

3

<b>238.Product of Array Except Self </b>  
Actually not two pointers, just loop  
Without using division, to complete O(N) that means can only loop the whole array constant times.  
Can construct a left array, stores the left product of each num in the array  
And another loop to construct a right array, stores the right product of each num in the array  
Then result of left * right for each position in the array.   
To optimize the space, we can use result array to store the left array at the first loop, the in second loop, construct var right, and directly apply it to the left array, which is result array, and update the result array on the fly.
a very good explanation in the first comment of this [post](https://leetcode.com/problems/product-of-array-except-self/discuss/65622/Simple-Java-solution-in-O(n)-without-extra-space)

a version with explanations of having the left and right array

In [46]:
def productExceptSelf(nums):
    n = len(nums)
    result, left, right = [0]*n, [0]*n, [0]*n
    # first num wont have left product, so default to 1, that means result[0]=1
    left[0] = 1   
    for i in range(1, n):
        # the first loop, construct left product for each n in nums
        left[i] = left[i-1] * nums[i-1]
    # last num wont have right product, so default to 1
    right[-1] = 1
    for i in range(n-2, -1, -1):
        # starting from back, construct the right product, and fill the result at one loop
        right[i] = right[i+1] * nums[i+1]
    for i in range(n):
        result[i] = left[i]*right[i]
    return result

In [47]:
nums = [1,2,3,4]
productExceptSelf(nums)

[24, 12, 8, 6]

this version is more concise from the above => same idea as above

In [51]:
from collections import Counter
def productExceptSelf(nums):
    n = len(nums)
    result = [0]*n
    # first num wont have left product, so default to 1
    result[0] = 1   
    for i in range(1, n):
        # the first loop, construct left product for each n in nums
        result[i] = result[i-1]*nums[i-1]
    # last num wont have right product, so default to 1
    right = 1
    for i in range(n-1, -1, -1):
        # starting from back, construct the right product, and fill the result at one loop
        # result = left prodct * right product, and we use result list to store the left product above
        result[i] *= right
        right *= nums[i]  # this is the right product for next value in nums
    return result

<b>76.Minimum Window Substring <b>

Sliding window (two pointers), first using right pointer to find spot where left and right together construct a window that includes all chars needed. Then move the left window to find the correct start to get rid of some unnecessary chars. Two sliders keep moving forward. Worst case left and right pointers all visited the whole array, but still time complexity is O(N)  
1. Use two pointers: start and end to represent a window.  
2. Move end to find a valid window.  
3. When a valid window is found, move start to find a smaller window.  

why start index j from 1?  
when set j to starting from 0, if the test case is s="a", t="aa", supposed to return empty string, but if set j to 0, the return would be s[start: end+1], which returns "a". Because end is from j, and it is a needed index, so need to +1 to ensure j is included in the result. But the s[start:end+1] wont work for the above test case. that means if using j start from 0, need special care of the cases before.   
if set j to start from 1, result is s[start:end], the above case would be s[0:0] which returns empty string, no need special care

In [48]:
def minWindow(s, t):
    """smart sliding window"""
    need = Counter(t)  # dict to see what chars should include. Magic about Counter dic is even c not in the dict, can still call dict[c] which will give 0 as the count
    missing = len(t) # how many chars are still needed
    # two pointers that save the smallest window found so far
    start, end = 0, 0 
    i = 0   # another pair of pointers i, j used to do sliding in each step of loop
    for j, char in enumerate(s, 1):  # start index j from 1
        if need[char] > 0:
            # find one needed char
            missing -=1
        # even char not in t, the counter dict will by default have it as count 0
        need[char] -= 1

        # if j slides to a position where all chars are found, move the i to update the window size, smaller but still holds what you need
        if missing == 0:
            # j starts from 1, 
            # a needed char starts count > 0, an unneeded char starts from 0
            # so when loop using i from start, if it is an unneeded char, the number is flipping back and i hits j, if it is a needed char before i hits j there is one splace need == 0
            while i < j and need[s[i]]<0:
                need[s[i]] +=1
                i += 1
            # now i points to a new start of a small sliding window that still holds every chars needed
            if end== 0 or j-i < end - start: 
                start, end = i, j # note, here end and j are the last needed char
            # now i need to move to the next position, the current position is a needed char, before moving, update the need and missing
            need[s[i]] +=1
            missing += 1
            i += 1
    return s[start: end]

In [52]:
s="a"
t="aa"
minWindow(s, t)

''

In [53]:
s = "ADOBECODEBANC"
t = "ABC"
minWindow(s, t)

'BANC'

<b> 763. Partition Labels </b>
actually three pointers.  
first construct a hashmap to store the right most index for each unique character
two pointers: start and end to mark the window of each partition
another pointer while loop the string as array, update the last index of the char by current pointer.
the `if i == end` is a smart step to find the right start

why last would be updated while looping?
for example: between those two occurrences of 'a', there could be other labels that make the minimum size of this partition bigger. 

time complexity O(N)


In [54]:
def partitionLabels(S):
    """three pointers actually"""
    # save the right most index of each char
    hashmap = {c: i for i, c in enumerate(S)}
    result = []
    # two pointers here
    start,end = 0, 0

    # construct another pointer
    for i, c in enumerate(S):
        end = max(end, hashmap[c])
        if i == end:
            result.append(end-start+1)
            start = i+1

    return result

In [55]:
S = "ababcbacadefegdehijhklij"
partitionLabels(S)

[9, 7, 8]

<b> 253.Meeting Rooms II </b>

In [59]:
def minMeetingRooms(intervals):
    starts = sorted(i[0] for i in intervals)
    ends = sorted(i[1] for i in intervals)
    available = room_needed = 0
    s = 0 # start meeting pointer
    e = 0  # end meeting pointer
    while s < len(intervals):
        # need a room
        if starts[s] < ends[e]:
            if available > 0:
                available -= 1
            else:
                room_needed += 1
            s+=1
        else:
            # a previous meeting is ended before the next one starts:
            # it will keep checking, and it works becauses when s == e, it will need a room
            available += 1
            e+=1
    return room_needed

In [60]:
intervals = [[0,30],[5,10],[15,20]]
minMeetingRooms(intervals)

2

In [61]:

intervals=[[0,2],[1,3],[2,4],[3,5],[4,6]]
minMeetingRooms(intervals)

2

<b>435.Non-overlapping Intervals</b>

sort by end is the key, keep the intervals ends first, so latter ones can have more room, so will reduce less amount of items compare with sort by start.

In [62]:
def eraseOverlapIntervals(intervals) :
    """find max number of items that are not overlapping"""
    if not intervals:
        return 0
     # sort by last number, so can find MIN number of intervals need to remove
        # keep the one ends early will give latter one more room to not overlap
    intervals.sort(key = lambda x: x[1]) 
    start = intervals[0]
    count = 1 # count is the non-overlapping count
    for end in intervals[1:]:
        if start[-1]<=end[0]:
            count += 1
            start = end
    return len(intervals)-count

In [63]:
intervals = [[1,2],[2,3],[3,4],[1,3]]
eraseOverlapIntervals(intervals)

1

another way, may be easier to understand

In [64]:
def eraseOverlapIntervals(intervals):
    """find max number of items that are not overlapping"""
    if not intervals:
        return 0
    # sort by last number, so can find MIN number of intervals need to remove

    # keep the one ends early will give latter one more room to not overlap
    intervals.sort(key = lambda x: x[1]) 
    end = float('-inf')
    result = 0
    for i in intervals:
        if i[0]>=end:
            # no overlapping, keep i
            end = i[1]
        else:
            # overlapping occurs
            result+=1
    return result

In [65]:
intervals = [[1,2],[2,3],[3,4],[1,3]]
eraseOverlapIntervals(intervals)

1

<b> 53. Maximum Subarray </b>

greedy way:  
current element = [-2, <b>1</b>, -3, <b>4</b>, -1, 2, 1, -5, 4]  
current max sum = [-2, <b>1</b>, -2, <b>4</b>, 3, 5, 6, 1, 5] bold num is where curr max restart  
max sum so far  = [-2, 1, 1, 4, 4, 5, 6, 6, 6]. => 6

loop the array once, so O(N)

In [ ]:
def maxSubArray(nums):
    curr_sum = max_sum = nums[0]
    for n in nums[1:]:
        curr_sum = max(curr_sum+n, n) # check where the sub array should restarts
        # restart the subarray at a better position, so curr_sum helps determine where it should start
        max_sum = max(max_sum, curr_sum)
        # control where the subarray should stop
    return max_sum

<b> 48 Rotate image </b>

way 1: first transpose (a[i][j] = a[j][i]). then reverse each row

In [1]:
def rotate(matrix):
    """
    Do not return anything, modify matrix in-place instead.
    """
    # transpose first, then reverse each row
    # transpose is litteraly a[i][j] = a[j][i]
    # imagine item in the diagonal line is steady, left and right of the diagonal line swapped. 
    # then remember to reverse each row to get the 90 degree rotation effect

    n = len(matrix)
    for i in range(n):
        for j in range(i, n):
            # start from the first column, never go back
            matrix[i][j], matrix[j][i] = matrix[j][i], matrix[i][j]

    for i in range(n):
        matrix[i]=matrix[i][::-1]

    return matrix

In [2]:
matrix = [[1,2,3],
  [4,5,6],
  [7,8,9]]
rotate(matrix)

[[7, 4, 1], [8, 5, 2], [9, 6, 3]]

<b> 415 Add Strings </b>

using while loop instead of for loop to avoid extra check of which array is longer after loop

using ord() to transform string numbers into integer numbers

In [6]:
def addStrings(num1, num2):
    result = []
    add_on = 0
    n1 = len(num1)-1
    n2 = len(num2)-1
    while n1>=0 or n2>=0:
        # in this way no need to check which array is longer after loop for extra elements
        x1 = ord(num1[n1])-ord('0') if n1>=0 else 0
        x2 = ord(num2[n2])-ord('0') if n2>=0 else 0
        value = (x1+x2+add_on)%10
        add_on = (x1+x2+add_on)//10
        result.append(value)
        n1-=1
        n2-=1
    # incase any add_on value left after the while loop
    if add_on:
        result.append(add_on)
    return ''.join(str(x) for x in result[::-1])


In [7]:
num1, num2 = "0", "0"
addStrings(num1, num2)

'0'